<a href="https://colab.research.google.com/github/ChoiDae1/Youtube_Project/blob/main/%EC%9C%A0%ED%8A%9C%EB%B8%8C_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [6]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [24]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import sys
import requests
import pandas as pd

In [73]:
it_과학기술 = {
    'ZHC': 'https://www.youtube.com/c/ZHcomicart/videos',
    'Mark Rober': 'https://www.youtube.com/c/MarkRober/videos',
    'Apple': 'https://www.youtube.com/user/Apple/videos',
    'Marques Brownlee': 'https://www.youtube.com/c/mkbhd/videos',
    'Veritasium': 'https://www.youtube.com/c/veritasium/videos',
    'CrazyRussianHacker': 'https://www.youtube.com/user/CrazyRussianHacker/videos',
    'SmarterEveryDay': 'https://www.youtube.com/c/smartereveryday/videos',
    'WIRED': 'https://www.youtube.com/channel/UCftwRNsjfRo08xYE31tkiyw',
    'EverythingApplePro E A P': 'https://www.youtube.com/c/EverythingApplePro/videos',
    '#Mind Warehouse': 'https://www.youtube.com/channel/UCYenDLnIHsoqQ6smwKXQ7Hg/videos',
    'JerryRigEverything': 'https://www.youtube.com/c/JerryRigEverything/videos',
    'iJustine': 'https://www.youtube.com/c/iJustine/videos',
    'hickok45': 'https://www.youtube.com/user/hickok45/videos',
    'freeCodeCamp.org': 'https://www.youtube.com/c/Freecodecamp/videos',
    'Austin Evans': 'https://www.youtube.com/c/austinevans/videos',
    'TheBackyardScientist': 'https://www.youtube.com/c/TheBackyardScientist/videos',
    'Seeker': 'https://www.youtube.com/c/Seeker/videos',
    'Ridddle': 'https://www.youtube.com/c/VineMontanaTV/videos',
    'Be Smart': 'https://www.youtube.com/c/itsokaytobesmart/videos',
    'The Organic Chemistry Tutor': 'https://www.youtube.com/c/TheOrganicChemistryTutor/videos'
              }
          

In [68]:
# 유튜버 한명 url(/videos로 끝남) 하나 받으면 최신순으로 영상 100개에 대한 크롤링 반환
def crawling_function(url):
    # 웹 드라이버 초기화
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    driver.implicitly_wait(5) # or bigger second
    
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    image_list = list() # 썸네일을 받을 수 있는 주소 저장용 리스트
    title_list = list() # 썸네일 제목을 저장하는 리스트
    view_list = list() # 조회수를 저장하는 리스트
    url_list = list() # url을 저장하는 리스트
    intro_list = list()

    idx = 1
    while True:

        try:
            img_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
            title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/h3/a'
            viewcnt_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/span[1]'
            
            # 이미지가 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
            img = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, img_xpath)))
            if img is None:
                print(idx, 'img is not loaded.')
            
            # 한 페이지에 약 8개 불러오는 데, 동영상 목록을 추가 불러오기 위해 스크롤 내림
            if idx % 8 == 0 :
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
            
            # 썸네일 주소를 리스트에 저장
            image = driver.find_element_by_xpath(img_xpath)
            img_url = image.get_attribute('src')
            image_list.append(img_url)

            # 타이틀을 리스트에 저장
            title = driver.find_element_by_xpath(title_xpath)
            title_list.append(title.text)

            # 조회수를 리스트에 저장
            view = driver.find_element_by_xpath(viewcnt_xpath)
            view_list.append(view.text)
            
            if idx == 100:
              break
            
            idx += 1

        except Exception as e:
            print()
            print(e)
            break

    #링크 크롤링
    page = driver.page_source
    soup = bs(page,'lxml')
    links = soup.find_all("div",id="dismissible")
    for link in links:
        url_list.append('https://www.youtube.com'+link.a['href'])
        if len(url_list) == 100:
          break

    # 설명 크롤링
    for link in url_list:
        driver.get(link)        
        time.sleep(5)
        page = driver.page_source
        soup = bs(page,'lxml')
        find = soup.find("span",class_="style-scope yt-formatted-string") 
        # find 객체가 None인 경우도 있기 때문에 이런 경우는 무시-> 설명을 None으로 대체
        if find == None:
            intro_list.append("None")
            continue

        spann = find.get_text()
        intro_list.append(spann)
    
    assert len(image_list) == len(title_list) == len(view_list) == len(url_list) == len(intro_list)
    driver.close()
    print('finish!')
    return image_list, title_list, view_list, url_list, intro_list

In [74]:
full_image_list = list()
full_title_list = list()
full_view_list = list()
full_url_list = list()
full_intro_list = list()

for youtuber, url in it_과학기술.items():
  image_list, title_list, view_list, url_list, intro_list = crawling_function(url)

dict_items([('ZHC', 'https://www.youtube.com/c/ZHcomicart/videos'), ('Mark Rober', 'https://www.youtube.com/c/MarkRober/videos'), ('Apple', 'https://www.youtube.com/user/Apple/videos'), ('Marques Brownlee', 'https://www.youtube.com/c/mkbhd/videos'), ('Veritasium', 'https://www.youtube.com/c/veritasium/videos'), ('CrazyRussianHacker', 'https://www.youtube.com/user/CrazyRussianHacker/videos'), ('SmarterEveryDay', 'https://www.youtube.com/c/smartereveryday/videos'), ('WIRED', 'https://www.youtube.com/channel/UCftwRNsjfRo08xYE31tkiyw'), ('EverythingApplePro E A P', 'https://www.youtube.com/c/EverythingApplePro/videos'), ('#Mind Warehouse', 'https://www.youtube.com/channel/UCYenDLnIHsoqQ6smwKXQ7Hg/videos'), ('JerryRigEverything', 'https://www.youtube.com/c/JerryRigEverything/videos'), ('iJustine', 'https://www.youtube.com/c/iJustine/videos'), ('hickok45', 'https://www.youtube.com/user/hickok45/videos'), ('freeCodeCamp.org', 'https://www.youtube.com/c/Freecodecamp/videos'), ('Austin Evans', 